In [14]:
import tqdm
import torch
import wandb
import numpy as np

import mdtraj as md
import nglview as nv

from tqdm import tqdm


n_particles = 22
n_dimensions = 3
scaling = 10
dim = n_particles * n_dimensions

In [22]:
# file_name = "tbg-v1"
file_name = "FM-AD2-original"
# file_name="Flow-Matching-AD2-amber-weighted-encoding-org"

In [25]:

npz_file = np.load(f"../result_data/{file_name}.npz")
latent_np=npz_file["latent_np"]
samples_np=npz_file["samples_np"]
dlogp_np=npz_file["dlogp_np"]
ad2_topology = md.load("../data/AD2/c5.pdb").topology
samples_np.shape

(200000, 66)

In [30]:
traj = md.Trajectory(
	xyz=(samples_np / scaling).reshape(-1, n_particles, n_dimensions),
	topology=ad2_topology
)

In [31]:
view = nv.show_mdtraj(traj)
view

NGLWidget(max_frame=199999)

In [13]:
nv.show_file("../data/AD2/c5.pdb")

NGLWidget()